In [ ]:
#NB: Some old code is commented out above replacement code
import flickrapi
import pandas as pd
import urllib.request
import time

In [ ]:
# These values have been removed and replaced with empty strings on the publicly posted version
api_key = ""
api_secret = ""
user_id = ""
file_path = ""

In [ ]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

In [ ]:
# Pulling list of photosets uploaded by ONHS, limited to one page of results at a time
# All API calls are preceded by a one-second sleep to ensure compliance with Flickr's 3600-per-hour call limit
# All API calls will attempt two retries in the case of an error
i = 3
while i:
    time.sleep(1)
    try:
        sets = flickr.photosets.getList(user_id=user_id, page=1)
    except:
        i -= 1
        continue
    else:
        break

In [ ]:
# Getting the total number of pages
pages = sets['photosets']['pages']

In [ ]:
# Converting results to DataFrame
picframe = pd.DataFrame(sets['photosets']['photoset'])

In [ ]:
# Getting the rest of the pages
for j in range(2, (pages+1)):
    i = 3
    while i:
        time.sleep(1)
        try:
            sets = flickr.photosets.getList(user_id=user_id, page=j)
        except:
            i -= 1
            continue
        else:
            break
    if i == 0:
        print(f"Sets call for page {j} exceeded max tries")
        continue
    picframe = picframe.append(pd.DataFrame(sets['photosets']['photoset']), ignore_index=True, sort=False)

In [ ]:
picframe.shape

In [ ]:
# Taking a look at the structure
picframe.head()

In [ ]:
# I was eliminating the _content dict wrapping the title, but this is now unnecessary
#picframe['title'] = [title['_content'] for title in picframe['title']]

In [ ]:
# Use known formatting scheme of titles to extract Olmsted firm job number - altered to be more robust, as 
# length of job number and positioning of pound sign are slightly inconsistent
#picframe['JobNum'] = [title[5:9] for title in picframe['title']]
picframe['JobNum'] = [title['_content'].split(',')[0].split('#')[1].strip() if title['_content'].find('Job') != -1 else "None" for title in picframe['title']]

In [ ]:
# Filter to get set IDs of desired job numbers - jobs can be excluded by commenting out the respective line
setIDs = picframe['id'][
    (picframe['JobNum'] == '290') | 
    (picframe['JobNum'] == '296') | 
    (picframe['JobNum'] == '297') | 
    (picframe['JobNum'] == '330') | 
    (picframe['JobNum'] == '623') | 
    (picframe['JobNum'] == '676') | 
    (picframe['JobNum'] == '1242') | 
    (picframe['JobNum'] == '1327') | 
    (picframe['JobNum'] == '1341') | 
    (picframe['JobNum'] == '2260') | 
    (picframe['JobNum'] == '2379') | 
    (picframe['JobNum'] == '2820') | 
    (picframe['JobNum'] == '2821') | 
    (picframe['JobNum'] == '2822') | 
    (picframe['JobNum'] == '2823') | 
    (picframe['JobNum'] == '2824') | 
    (picframe['JobNum'] == '2825') | 
    (picframe['JobNum'] == '2826') | 
    (picframe['JobNum'] == '2827') | 
    (picframe['JobNum'] == '2828') | 
    (picframe['JobNum'] == '2829') | 
    (picframe['JobNum'] == '2830') | 
    (picframe['JobNum'] == '2831') | 
    (picframe['JobNum'] == '2832') | 
    (picframe['JobNum'] == '2833') | 
    (picframe['JobNum'] == '2834') | 
    (picframe['JobNum'] == '2835') | 
    (picframe['JobNum'] == '2836') | 
    (picframe['JobNum'] == '2837') | 
    (picframe['JobNum'] == '2838') | 
    (picframe['JobNum'] == '2839') | 
    (picframe['JobNum'] == '2840') | 
    (picframe['JobNum'] == '2841') | 
    (picframe['JobNum'] == '2842') | 
    (picframe['JobNum'] == '2843') |
    (picframe['JobNum'] == '2844') | 
    (picframe['JobNum'] == '2845') |
    (picframe['JobNum'] == '2846') |
    (picframe['JobNum'] == '2847') |
    (picframe['JobNum'] == '2848') |
    (picframe['JobNum'] == '2849') |
    (picframe['JobNum'] == '2901') |
    (picframe['JobNum'] == '2914') |
    (picframe['JobNum'] == '2922') |
    (picframe['JobNum'] == '2931') |
    (picframe['JobNum'] == '2956') |
    (picframe['JobNum'] == '2961') |
    (picframe['JobNum'] == '2961') |
    (picframe['JobNum'] == '2964') |
    (picframe['JobNum'] == '2966') |
    (picframe['JobNum'] == '3035') |
    (picframe['JobNum'] == '3086') |
    (picframe['JobNum'] == '3297') |
    (picframe['JobNum'] == '3346') |
    (picframe['JobNum'] == '3358') |
    (picframe['JobNum'] == '3604') |
    (picframe['JobNum'] == '3643') |
    (picframe['JobNum'] == '3706') |
    (picframe['JobNum'] == '3749') |
    (picframe['JobNum'] == '5358') |
    (picframe['JobNum'] == '5518') |
    (picframe['JobNum'] == '5507') |
    (picframe['JobNum'] == '6487') |
    (picframe['JobNum'] == '6570') |
    (picframe['JobNum'] == '6574') |
    (picframe['JobNum'] == '6790') |
    (picframe['JobNum'] == '7002') |
    (picframe['JobNum'] == '7022') |
    (picframe['JobNum'] == '7054') |
    (picframe['JobNum'] == '7290') |
    (picframe['JobNum'] == '7308') |
    (picframe['JobNum'] == '7458') |
    (picframe['JobNum'] == '7714') |
    (picframe['JobNum'] == '7956') |
    (picframe['JobNum'] == '9299') |
    (picframe['JobNum'] == '9300') |
    (picframe['JobNum'] == '9451') |
    (picframe['JobNum'] == '9970') |
    (picframe['JobNum'] == '9992') |
    (picframe['JobNum'] == '12034') |
    (picframe['JobNum'] == '12044')
]

In [ ]:
# Get photo list from each set and extract photo IDs, compiling list of all photoIDs. Sleep added to limit API load
photoIDs = []
for setID in setIDs:
    i = 3
    while i:
        time.sleep(1)
        try:
            photos = flickr.photosets.getPhotos(api_key=api_key, photoset_id=setID, user_id=user_id)
        except:
            i -= 1
            continue
        else:
            break
    if i == 0:
        print(f"Set ID {setID} exceeded max tries")
        continue
    photoList = photos['photoset']['photo']
    for photo in photoList:
        photoIDs.append(photo['id'])

In [ ]:
# Get file sources from photos, selecting for size by position in size list. -1 is always the original file. 
# For loop and sleep added to limit load. 
#urlList = [flickr.photos.getSizes(api_key=api_key, photo_id=photoID)['sizes']['size'][0]['source'] for photoID in photoIDs]
urlList = []
for photoID in photoIDs:
    i = 3
    while i:
        time.sleep(1)
        try:
            sizeDict = flickr.photos.getSizes(api_key=api_key, photo_id=photoID)
        except:
            i -= 1
            continue
        else:
            break
    if i == 0:
        print(f"Photo ID source call {photoID} exceeded max tries")
        continue
    urlList.append(sizeDict['sizes']['size'][-1]['source'])

In [ ]:
# Iterate through sources, saving files named as the corresponding photo ID + .jpg
j = 0
while j < len(urlList):
    i = 3
    while i:
        time.sleep(1)
        try:
            urllib.request.urlretrieve(urlList[j], f'{file_path}/{photoIDs[j]}.jpg')
        except:
            i -= 1
            continue
        else:
            break
    if i == 0:
        print(f"URL {urlList[j]} exceeded max tries, photo {photoIDs[j]} not retrieved")
        continue
    j += 1

In [ ]:
# Now to create a metadata file...
infoList = []

# Pulling the info for each photo and getting the description content
for photoID in photoIDs:
    i = 3
    while i:
        time.sleep(1)
        try:
            info = flickr.photos.getInfo(api_key=api_key, photo_id=photoID)['photo']['description']['_content']
        except:
            i -= 1
            continue
        else:
            break
    if i == 0:
        print(f"Photo ID {photoID} info call exceeded max tries")
        continue

    # Using known formatting of description to create metadata spreadsheet. Splitting info into lines, eliminating newlines at colons
    info = info.replace(':\n', ':')
    info = info.split('\n')

    # Slicing off the attribution lines (proper attribution will be given, though!)
    info = info[:-2]

    # Turning each line into a key-value pair and adding to a dict, then appending dict to list
    # Initializing dict with photoID as "id"
    infoDict = {'id': photoID}
    for inf in info:
        inf = inf.split(':', 2)
        if len(inf) < 2:
            continue
        infoDict[inf[0]] = inf[1]
    infoList.append(infoDict)

In [ ]:
# Converting list of dictionaries to dataframe
infoframe = pd.DataFrame(infoList)

In [ ]:
# Let's output that as a .csv
infoframe.to_csv(f'{file_path}/metadata.csv', index=False)